In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import math
from tqdm import tqdm
from functions import *
from tickers_list import *
sns.set_style('whitegrid')

In [3]:
stocks = ["LUPIN.NS", "VOLTAS.NS", "YESBANK.NS"]

INITIAL_AMT_INVESTED = 10000
NUM_OF_SIMULATIONS = 5000
start_date='2023-01-01'
end_date='2023-12-31'

In [5]:
## GATHER STOCK PRICES

stock_prices, stock_dividend = fetch_data(stocks, start_date='2023-01-01', end_date='2023-12-31')

## CALCULATE LOG RETURNS

log_returns = np.log(stock_prices/stock_prices.shift(1))
log_returns = log_returns[1:]

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  5.16it/s]


In [6]:
## DOWNSIDE DEVIATION (SORTINO RATIO)

log_returns_down_only = log_returns.applymap(lambda x: 0 if x>0 else x)

In [185]:
portfolio_weights = []
portfolio_num_of_shares = []
portfolio_returns = []
portfolio_risks = []
portfolio_risks_sortino = []
portfolio_value = []
portfolio_beta = []
NUM_TRADING_DAYS = len(stock_prices)

nifty50_price,temp = fetch_data("^NSEI", start_date=start_date, end_date=end_date)
log_returns_market = np.log(nifty50_price/nifty50_price.shift(1))[1:]

## MONTE CARLO SIMULATION

for i in tqdm(range(NUM_OF_SIMULATIONS)):
    ## STEP-I: generating random weights
    w = np.random.rand(log_returns.shape[1])
    w = np.expand_dims(w, axis=0)
    w = w/np.sum(w)
    portfolio_weights.append(w.flatten())

    ## STEP-II: calculating returns using average annual returns
    p_returns = (log_returns.mean() @ w.T*NUM_TRADING_DAYS)[0]
    portfolio_returns.append(p_returns)

    ## STEP-III: calculating risks
    p_risks = np.sqrt(w @ log_returns.cov()*NUM_TRADING_DAYS @ w.T)[0][0]
    portfolio_risks.append(p_risks)
    
    p_risks = np.sqrt(w @ log_returns_down_only.cov()*NUM_TRADING_DAYS @ w.T)[0][0]
    portfolio_risks_sortino.append(p_risks)

    ## CALCULATING PORTFOLIO VALUE
    num_of_shares = INITIAL_AMT_INVESTED*w[0]/stock_prices.iloc[0,:]
    num_of_shares = np.expand_dims(num_of_shares, axis=0)
    portfolio_num_of_shares.append(num_of_shares)

    portfolio_value_i = np.sum(num_of_shares*stock_prices, axis=1)
    portfolio_value.append(portfolio_value_i)

    log_returns_portfolio_value_i = np.log(portfolio_value_i/portfolio_value_i.shift(1))[1:]

    ## STEP-IV: calculating portfolio beta
    beta = calculate_beta(log_returns_portfolio_value_i, log_returns_market)
    portfolio_beta.append(beta)

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:11<00:00, 448.19it/s]


In [186]:
portfolio_weights = np.array(portfolio_weights)
portfolio_num_of_shares = np.array(portfolio_num_of_shares)
portfolio_returns = np.array(portfolio_returns)
portfolio_risks = np.array(portfolio_risks)
portfolio_risks_sortino = np.array(portfolio_risks_sortino)
portfolio_value = np.array(portfolio_value)
portfolio_beta = np.array(portfolio_beta)



# taking IR of government issued 10-year bonds as risk-free rate
risk_free_rate = 0.07

## CALCULATING RATIOS

sharpe_ratios = (portfolio_returns - risk_free_rate) / portfolio_risks
sortino_ratio = (portfolio_returns - risk_free_rate) / portfolio_risks_sortino

In [159]:
## OPTIMIZED WEIGHTS

index = np.argmax(sortino_ratios)
optimized_weights = portfolio_weights[index]

num_of_shares = INITIAL_AMT_INVESTED*optimized_weights/stock_prices.iloc[0,:]
num_of_shares = np.expand_dims(num_of_shares, axis=0)

In [106]:
## CALCULATING PORTFOLIO VALUE
portfolio_value = np.sum(num_of_shares*np.array(stock_prices), axis=1)

## PORTFOLIO DATAFRAME
df_portfolio = num_of_shares*stock_prices
df_portfolio['PORTFOLIO_VALUE'] = portfolio_value

In [91]:
## FUNCTION TO CONVERT WEIGHTS TO DATAFRAME

def weights_to_dataframe(arr, cols):
    
    if arr.ndim != 2:
        raise ValueError("Input error must have only 2 dimensions")
    
    for i in range(arr.shape[1]):
        if i==0:
            df = pd.DataFrame(arr[:,i])
        else:
            temp = pd.DataFrame(arr[:,i])
            df = pd.concat([df,temp], axis=1)
    df.columns = cols
    return df

In [93]:
## CREATING DATAFRAME FOR SIMULATED WEIGHTS

df_sims = pd.DataFrame({'Portfolio_return':portfolio_returns,'Portfolio_risk':portfolio_risks,'Sortino':sortino_ratios})
df_sims = pd.concat([df_sims,weights_to_dataframe(portfolio_weights, stocks)], axis=1)
df_sims = df_sims.sort_values(by=['Sortino'], ascending=False)

In [155]:
def calculate_beta(log_return_portfolio, log_return_market):
    
    covariance = np.cov(log_return_portfolio, log_return_market.iloc[:,0])[0,1]
    variance_market = log_return_market.var()
    
    beta = covariance/variance_market
    
    return beta[0]

In [156]:
nifty50_price,temp = fetch_data("^NSEI", start_date=start_date, end_date=end_date)
log_returns_market = np.log(nifty50_price/nifty50_price.shift(1))[1:]

log_returns_portfolio = np.log(df_portfolio['PORTFOLIO_VALUE']/df_portfolio['PORTFOLIO_VALUE'].shift(1))[1:]

beta = calculate_beta(log_returns_portfolio, log_returns_market)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.19it/s]


In [180]:
w[0]*INITIAL_AMT_INVESTED/stock_prices.iloc[0,:]

LUPIN.NS       6.758875
VOLTAS.NS      4.279608
YESBANK.NS    76.265819
Name: 2023-01-02 00:00:00+05:30, dtype: float64

In [182]:
portfolio_weights = []
portfolio_num_of_shares = []
portfolio_returns = []
portfolio_risks = []
portfolio_value = []
portfolio_beta = []
NUM_TRADING_DAYS = len(stock_prices)

nifty50_price,temp = fetch_data("^NSEI", start_date=start_date, end_date=end_date)
log_returns_market = np.log(nifty50_price/nifty50_price.shift(1))[1:]

## MONTE CARLO SIMULATION


## STEP-I: generating random weights
w = np.random.rand(log_returns.shape[1])
w = np.expand_dims(w, axis=0)
w = w/np.sum(w)
portfolio_weights.append(w.flatten())
    
## STEP-II: calculating returns using average annual returns
p_returns = (log_returns.mean() @ w.T*NUM_TRADING_DAYS)[0]
portfolio_returns.append(p_returns)
    
## STEP-III: calculating risks
p_risks = np.sqrt(w @ log_returns_down_only.cov()*NUM_TRADING_DAYS @ w.T)[0][0]
portfolio_risks.append(p_risks)
    
## CALCULATING PORTFOLIO VALUE
num_of_shares = INITIAL_AMT_INVESTED*w[0]/stock_prices.iloc[0,:]
num_of_shares = np.expand_dims(num_of_shares, axis=0)
portfolio_num_of_shares.append(num_of_shares)

portfolio_value_i = np.sum(num_of_shares*stock_prices, axis=1)
portfolio_value.append(portfolio_value_i)

log_returns_portfolio_value_i = np.log(portfolio_value_i/portfolio_value_i.shift(1))[1:]

## STEP-IV: calculating portfolio beta
beta = calculate_beta(log_returns_portfolio_value_i, log_returns_market)
portfolio_beta.append(beta)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.77it/s]
